# Final Assignment — Tesla & GameStop (Q1–Q7)


In [1]:
# === Setup / Install ===
# If some libraries are missing, uncomment the line below and run once:
!pip install pandas yfinance requests beautifulsoup4 lxml plotly kaleido prophet statsmodels
!pip install -U "plotly>=6.1.1"

import pandas as pd
import yfinance as yf
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go

# Browser header for web scraping (important for macrotrends.net)
HEADERS = {"User-Agent": "Mozilla/5.0"}

print("✅ Libraries imported and headers set.")

✅ Libraries imported and headers set.


## Question 1: Extracting Tesla Stock Data Using yfinance

Reset the index, save, and display the first five rows of the tesla_data dataframe using the head function. Upload a screenshot of the results and code from the beginning of Question 1 to the results below.

In [2]:
# Q1 — Tesla with yfinance (single output)
tesla = yf.Ticker("TSLA")
tesla_data = tesla.history(period="max").reset_index()
tesla_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29 00:00:00-04:00,1.266667,1.666667,1.169333,1.592667,281494500,0.0,0.0
1,2010-06-30 00:00:00-04:00,1.719333,2.028000,1.553333,1.588667,257806500,0.0,0.0
2,2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000,0.0,0.0
3,2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000,0.0,0.0
4,2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500,0.0,0.0


## Question 2: Extracting Tesla Revenue Data Using Webscraping

Display the last five rows of the tesla_revenue dataframe using the tail function. Upload a screenshot of the results.

In [3]:
# Q2 — Tesla revenue (minimal, handles Year vs Date)

from io import StringIO

url = "https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue"

# Fetch HTML with headers to avoid 403
resp = requests.get(
    url,
    headers={
        "User-Agent": "Mozilla/5.0",
        "Accept-Language": "en-US,en;q=0.9",
        "Referer": "https://www.google.com",
    },
    timeout=30
)
resp.raise_for_status()

# Parse all tables from the page HTML
tables = pd.read_html(StringIO(resp.text), flavor="lxml")

def normalize_rev_table(df: pd.DataFrame) -> pd.DataFrame | None:
    # Try to locate Date/Revenue (or Year/Revenue) in a compact way
    cols_lower = [str(c).strip().lower() for c in df.columns]
    if "date" in cols_lower and "revenue" in cols_lower:
        dcol = df.columns[cols_lower.index("date")]
        rcol = df.columns[cols_lower.index("revenue")]
        out = df[[dcol, rcol]].copy()
        out.columns = ["Date", "Revenue"]
    elif "year" in cols_lower and "revenue" in cols_lower:
        ycol = df.columns[cols_lower.index("year")]
        rcol = df.columns[cols_lower.index("revenue")]
        out = df[[ycol, rcol]].copy()
        out.columns = ["Date", "Revenue"]   # convert 4-digit years below
    else:
        # Fallback: assume first two columns are Date-like and Revenue-like
        if df.shape[1] < 2:
            return None
        out = df.iloc[:, :2].copy()
        out.columns = ["Date", "Revenue"]

    # Clean date: if values are 4-digit years, map to YYYY-12-31; else parse normally
    s = out["Date"].astype(str).str.strip()
    is_year = s.str.fullmatch(r"\d{4}")
    out["Date"] = pd.to_datetime(
        s.where(~is_year, s + "-12-31"),
        errors="coerce"
    )
    out = out.dropna(subset=["Date"])

    # Clean revenue -> float
    out["Revenue"] = (
        out["Revenue"].astype(str)
        .str.replace(r"[\$,]", "", regex=True)
        .str.strip()
    )
    out = out[out["Revenue"] != ""].copy()
    out["Revenue"] = pd.to_numeric(out["Revenue"], errors="coerce")
    out = out.dropna(subset=["Revenue"]).sort_values("Date").reset_index(drop=True)
    return out if not out.empty else None

# Pick the first usable revenue table
tesla_revenue = None
for t in tables:
    cand = normalize_rev_table(t)
    if cand is not None:
        tesla_revenue = cand
        break

if tesla_revenue is None:
    raise RuntimeError("Revenue table not found. Re-run the cell.")
tesla_revenue.tail(5)

,Date,Revenue
11,2020-12-31,31536
12,2021-12-31,53823
13,2022-12-31,81462
14,2023-12-31,96773
15,2024-12-31,97690


## Question 3: Extracting GameStop Stock Data Using yfinance

Display the last five rows of the tesla_revenue dataframe using the tail function. Upload a screenshot of the results.

In [4]:
gme = yf.Ticker("GME")
gme_data = gme.history(period="max")
gme_data.reset_index(inplace=True)
gme_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2002-02-13 00:00:00-05:00,1.620128,1.693350,1.603296,1.691667,76216000,0.0,0.0
1,2002-02-14 00:00:00-05:00,1.712707,1.716074,1.670626,1.683250,11021600,0.0,0.0
2,2002-02-15 00:00:00-05:00,1.683251,1.687459,1.658002,1.674834,8389600,0.0,0.0
3,2002-02-19 00:00:00-05:00,1.666418,1.666418,1.578047,1.607504,7410400,0.0,0.0
4,2002-02-20 00:00:00-05:00,1.615920,1.662210,1.603296,1.662210,6892800,0.0,0.0


## Question 4: Extracting GameStop Revenue Data Using Webscraping

Display the last five rows of the gme_revenue dataframe using the tail function. Upload a screenshot of the results.

In [5]:
# Q4 — GameStop revenue (minimal, 403-safe, handles Year vs Date)

from io import StringIO

url = "https://www.macrotrends.net/stocks/charts/GME/gamestop/revenue"

# fetch HTML with headers to avoid 403
resp = requests.get(
    url,
    headers={
        "User-Agent": "Mozilla/5.0",
        "Accept-Language": "en-US,en;q=0.9",
        "Referer": "https://www.google.com",
    },
    timeout=30
)
resp.raise_for_status()

# parse all tables from HTML string
tables = pd.read_html(StringIO(resp.text), flavor="lxml")

def normalize_rev_table(df: pd.DataFrame) -> pd.DataFrame | None:
    cols_lower = [str(c).strip().lower() for c in df.columns]
    if "date" in cols_lower and "revenue" in cols_lower:
        dcol = df.columns[cols_lower.index("date")]
        rcol = df.columns[cols_lower.index("revenue")]
        out = df[[dcol, rcol]].copy()
        out.columns = ["Date", "Revenue"]
    elif "year" in cols_lower and "revenue" in cols_lower:
        ycol = df.columns[cols_lower.index("year")]
        rcol = df.columns[cols_lower.index("revenue")]
        out = df[[ycol, rcol]].copy()
        out.columns = ["Date", "Revenue"]
    else:
        if df.shape[1] < 2:
            return None
        out = df.iloc[:, :2].copy()
        out.columns = ["Date", "Revenue"]

    s = out["Date"].astype(str).str.strip()
    is_year = s.str.fullmatch(r"\d{4}")
    out["Date"] = pd.to_datetime(s.where(~is_year, s + "-12-31"), errors="coerce")
    out = out.dropna(subset=["Date"])

    out["Revenue"] = (
        out["Revenue"].astype(str)
        .str.replace(r"[\$,]", "", regex=True)
        .str.strip()
    )
    out = out[out["Revenue"] != ""].copy()
    out["Revenue"] = pd.to_numeric(out["Revenue"], errors="coerce")
    out = out.dropna(subset=["Revenue"]).sort_values("Date").reset_index(drop=True)
    return out if not out.empty else None

# pick the first usable revenue table
gme_revenue = None
for t in tables:
    cand = normalize_rev_table(t)
    if cand is not None:
        gme_revenue = cand
        break

if gme_revenue is None:
    raise RuntimeError("Revenue table not found. Re-run the cell.")

# single output line for the screenshot
gme_revenue.tail(5)

,Date,Revenue
12,2021-12-31,5090
13,2022-12-31,6011
14,2023-12-31,5927
15,2024-12-31,5273
16,2025-12-31,3823


## Question 5: Tesla Stock and Revenue Dashboard

Use the make_graph function to graph the Tesla Stock Data, also provide a title for the graph.

Upload a screenshot of your results.

In [18]:
# Q5 — Tesla: stock vs revenue (single graph)

import plotly.io as pio
pio.renderers.default = "iframe"  # reliable inline renderer

# prepare data (light normalization)
ts = tesla_data.sort_values("Date").copy()
ts["Date"] = pd.to_datetime(ts["Date"], errors="coerce", utc=True).dt.tz_localize(None)

tr = tesla_revenue.sort_values("Date").copy()
tr["Date"] = pd.to_datetime(tr["Date"], errors="coerce")
tr["Revenue"] = pd.to_numeric(tr["Revenue"], errors="coerce")

fig = go.Figure()
fig.add_trace(go.Scatter(x=ts["Date"], y=ts["Close"], mode="lines", name="Tesla Close"))
fig.add_trace(go.Scatter(x=tr["Date"], y=tr["Revenue"], mode="lines+markers",
                         name="Tesla Revenue", yaxis="y2"))

fig.update_layout(
    title="Tesla: Stock Price vs Quarterly Revenue",
    xaxis_title="Date",
    yaxis=dict(title="Close (USD)"),
    yaxis2=dict(title="Revenue (USD)", overlaying="y", side="right"),
    xaxis_rangeslider=dict(visible=False),
    height=650
)

fig.show()

# fallback: also save HTML for screenshot if inline is hidden
fig.write_html("tesla_graph.html", include_plotlyjs="cdn")


## Question 6:  GameStop Stock and Revenue Dashboard

Use the make_graph function to graph the GameStop Stock Data, also provide a title for the graph.

Upload a screenshot of your results.

In [20]:
# Q6 — GameStop: stock vs revenue (single graph)

import plotly.io as pio
pio.renderers.default = "iframe"  # inline renderer (same as Q5)

# prepare data (light normalization)
gs = gme_data.sort_values("Date").copy()
gs["Date"] = pd.to_datetime(gs["Date"], errors="coerce", utc=True).dt.tz_localize(None)

gr = gme_revenue.sort_values("Date").copy()
gr["Date"] = pd.to_datetime(gr["Date"], errors="coerce")
gr["Revenue"] = pd.to_numeric(gr["Revenue"], errors="coerce")

fig = go.Figure()
fig.add_trace(go.Scatter(x=gs["Date"], y=gs["Close"], mode="lines", name="GameStop Close"))
fig.add_trace(go.Scatter(x=gr["Date"], y=gr["Revenue"], mode="lines+markers",
                         name="GameStop Revenue", yaxis="y2"))

fig.update_layout(
    title="GameStop: Stock Price vs Quarterly Revenue",
    xaxis_title="Date",
    yaxis=dict(title="Close (USD)"),
    yaxis2=dict(title="Revenue (USD)", overlaying="y", side="right"),
    xaxis_rangeslider=dict(visible=False),
    height=650
)

fig.show()

# fallback: also save HTML for screenshot if inline is hidden
fig.write_html("gme_graph.html", include_plotlyjs="cdn")


## Question 7:  Sharing your Assignment Notebook

Add the GitHub link or the URL to your assignment in Watson Studio using the share notebook lab instructions.

In [ ]:
links={
 'GitHub':'https://github.com/<username>/IBM_DS_Final_Assignment',
 'Watson_Studio':'<Watson Studio URL>',
 'Cognos_Dashboard':'<Cognos URL>',
 'Tableau_Public':'<Tableau URL>'
}
links

## 8(a): Forecast TSLA Monthly Close

In [21]:
# 8(a): ====== Forecast TSLA Monthly Close ======
from datetime import timedelta
tsla_monthly=tesla_data.copy()
tsla_monthly["Date"]=pd.to_datetime(tsla_monthly["Date"],errors="coerce",utc=True).dt.tz_convert("UTC").dt.tz_localize(None)
tsla_monthly=tsla_monthly.set_index("Date")["Close"].resample("M").last().dropna()
dfp=tsla_monthly.reset_index().rename(columns={"Date":"ds","Close":"y"})
horizon=12
forecast_df=None
try:
    from prophet import Prophet
    m=Prophet(); m.fit(dfp)
    future=m.make_future_dataframe(periods=horizon,freq="M")
    fcst=m.predict(future)
    forecast_df=fcst[["ds","yhat","yhat_lower","yhat_upper"]].tail(horizon)
except Exception as e:
    import statsmodels.api as sm
    model=sm.tsa.ARIMA(dfp["y"],order=(1,1,1))
    res=model.fit()
    pred=res.get_forecast(steps=horizon)
    fc=pred.summary_frame()
    import pandas as pd
    last= dfp["ds"].max()
    dates=pd.date_range(last+pd.offsets.MonthEnd(1),periods=horizon,freq="M")
    forecast_df=pd.DataFrame({"ds":dates,"yhat":fc["mean"],"yhat_lower":fc["mean_ci_lower"],"yhat_upper":fc["mean_ci_upper"]})
forecast_df.head()

INFO	prophet:forecaster.py:parse_seasonality_args()- Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO	prophet:forecaster.py:parse_seasonality_args()- Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG	cmdstanpy:filesystem.py:_temp_single_json()- input tempfile: /tmp/wsuser/tmpxv6ard6v/t60j7ez4.json
DEBUG	cmdstanpy:filesystem.py:_temp_single_json()- input tempfile: /tmp/wsuser/tmpxv6ard6v/j02y2tle.json
DEBUG	cmdstanpy:model.py:_run_cmdstan()- idx 0
DEBUG	cmdstanpy:model.py:_run_cmdstan()- running CmdStan, num_threads: None
DEBUG	cmdstanpy:model.py:_run_cmdstan()- CmdStan args: ['/opt/conda/envs/Python-RT24.1/lib/python3.11/site-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=65868', 'data', 'file=/tmp/wsuser/tmpxv6ard6v/t60j7ez4.json', 'init=/tmp/wsuser/tmpxv6ard6v/j02y2tle.json', 'output', 'file=/tmp/wsuser/tmpxv6ard6v/prophet_modeli7z_bg7a/prophet_model-20250921085137.csv', 'method=o

,ds,yhat,yhat_lower,yhat_upper
184,2025-10-31,333.855626,270.053785,394.556583
185,2025-11-30,348.235854,283.657867,410.612911
186,2025-12-31,352.458319,286.955260,412.462307
187,2026-01-31,354.872664,294.538009,419.888172
188,2026-02-28,348.469343,283.741184,409.054396


In [22]:
import plotly.graph_objects as go
fig=go.Figure()
fig.add_trace(go.Scatter(x=dfp["ds"],y=dfp["y"],mode="lines",name="History"))
if forecast_df is not None:
    fig.add_trace(go.Scatter(x=forecast_df["ds"],y=forecast_df["yhat"],mode="lines+markers",name="Forecast"))
fig.update_layout(title="TSLA Monthly Close Forecast",height=600)
fig.show()

## 8(b): GameStop Monthly Close

In [25]:
# 8b === Forecast GME Monthly Close ===

gme_monthly = gme_data.copy()
gme_monthly["Date"] = pd.to_datetime(gme_monthly["Date"], errors="coerce", utc=True).dt.tz_localize(None)
gme_monthly = gme_monthly.set_index("Date")["Close"].resample("M").last().dropna()

dfp_gme = gme_monthly.reset_index().rename(columns={"Date":"ds","Close":"y"})
horizon = 12
forecast_gme = None

try:
    from prophet import Prophet
    m = Prophet()
    m.fit(dfp_gme)
    future = m.make_future_dataframe(periods=horizon, freq="M")
    fcst = m.predict(future)
    forecast_gme = fcst[["ds","yhat","yhat_lower","yhat_upper"]].tail(horizon)
except Exception as e:
    import statsmodels.api as sm
    model = sm.tsa.ARIMA(dfp_gme["y"], order=(1,1,1))
    res = model.fit()
    pred = res.get_forecast(steps=horizon)
    fc = pred.summary_frame()
    last = dfp_gme["ds"].max()
    dates = pd.date_range(last+pd.offsets.MonthEnd(1), periods=horizon, freq="M")
    forecast_gme = pd.DataFrame({
        "ds": dates,
        "yhat": fc["mean"],
        "yhat_lower": fc["mean_ci_lower"],
        "yhat_upper": fc["mean_ci_upper"]
    })

forecast_gme.head()


INFO	prophet:forecaster.py:parse_seasonality_args()- Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO	prophet:forecaster.py:parse_seasonality_args()- Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG	cmdstanpy:filesystem.py:_temp_single_json()- input tempfile: /tmp/wsuser/tmpxv6ard6v/_0d3efjy.json
DEBUG	cmdstanpy:filesystem.py:_temp_single_json()- input tempfile: /tmp/wsuser/tmpxv6ard6v/3vaknbew.json
DEBUG	cmdstanpy:model.py:_run_cmdstan()- idx 0
DEBUG	cmdstanpy:model.py:_run_cmdstan()- running CmdStan, num_threads: None
DEBUG	cmdstanpy:model.py:_run_cmdstan()- CmdStan args: ['/opt/conda/envs/Python-RT24.1/lib/python3.11/site-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=36927', 'data', 'file=/tmp/wsuser/tmpxv6ard6v/_0d3efjy.json', 'init=/tmp/wsuser/tmpxv6ard6v/3vaknbew.json', 'output', 'file=/tmp/wsuser/tmpxv6ard6v/prophet_model1r37zdw0/prophet_model-20250921090353.csv', 'method=o

,ds,yhat,yhat_lower,yhat_upper
284,2025-10-31,28.398762,17.907182,38.881386
285,2025-11-30,28.886079,18.175474,40.239224
286,2025-12-31,28.311515,17.520576,39.367747
287,2026-01-31,31.739689,21.503832,42.006594
288,2026-02-28,29.017361,17.727938,39.588472


In [35]:
# === Extra 8b — Plot GME forecast ===

fig = go.Figure()

# Actual monthly close prices
fig.add_trace(go.Scatter(
    x=dfp_gme["ds"], y=dfp_gme["y"],
    mode="lines", name="GME Close (actual)"
))

# Forecasted values (central estimate)
fig.add_trace(go.Scatter(
    x=forecast_gme["ds"], y=forecast_gme["yhat"],
    mode="lines+markers", name="Forecast"
))

# Confidence interval (shaded area)
fig.add_trace(go.Scatter(
    x=forecast_gme["ds"].tolist() + forecast_gme["ds"].tolist()[::-1],
    y=forecast_gme["yhat_upper"].tolist() + forecast_gme["yhat_lower"].tolist()[::-1],
    fill="toself",
    fillcolor="rgba(0,100,80,0.2)",
    line=dict(color="rgba(255,255,255,0)"),
    hoverinfo="skip",
    showlegend=True,
    name="Confidence Interval"
))

fig.update_layout(
    title="GameStop Forecast (Monthly Close)",
    xaxis_title="Date",
    yaxis_title="Price (USD)",
    height=600
)

fig.show()


## 9: Export CSVs -- monthly history, forecasts, Tableau long-format.

In [34]:
# === One-cell export pack: CSVs + Tableau long-format (no warnings) + HTML figures ===
import pandas as pd
from pathlib import Path
import plotly.graph_objects as go

# ---------------- paths ----------------
out_data = Path("./data")
out_figs = Path("./figures")
out_data.mkdir(parents=True, exist_ok=True)
out_figs.mkdir(parents=True, exist_ok=True)

# ---------------- helpers ----------------
def to_monthly_close(df: pd.DataFrame, symbol: str) -> pd.DataFrame:
    """Resample to month-end Close; returns [Symbol, Date, Close]."""
    s = df.copy()
    s["Date"] = pd.to_datetime(s["Date"], errors="coerce", utc=True).dt.tz_localize(None)
    s = s.set_index("Date")["Close"].resample("M").last().dropna().reset_index()
    s.insert(0, "Symbol", symbol)
    s.columns = ["Symbol", "Date", "Close"]
    return s

def fmt_forecast(df: pd.DataFrame | None, symbol: str) -> pd.DataFrame:
    """Normalize Prophet/ARIMA forecast to [Symbol, Date, Forecast, Lower, Upper]."""
    if df is None or df.empty:
        return pd.DataFrame(columns=["Symbol","Date","Forecast","Lower","Upper"])
    out = df.rename(columns={"ds":"Date","yhat":"Forecast","yhat_lower":"Lower","yhat_upper":"Upper"}).copy()
    out.insert(0, "Symbol", symbol)
    return out[["Symbol","Date","Forecast","Lower","Upper"]]

def actual_long(df_hist: pd.DataFrame) -> pd.DataFrame:
    """Convert monthly history to Tableau long-format [Symbol, Date, Series, Value, Lower, Upper]."""
    d = df_hist.rename(columns={"Close":"Value"}).copy()
    d["Series"] = "Actual"
    d["Lower"] = pd.NA
    d["Upper"] = pd.NA
    return d[["Symbol","Date","Series","Value","Lower","Upper"]]

def forecast_long(df_fc: pd.DataFrame) -> pd.DataFrame:
    """Convert normalized forecast to Tableau long-format rows."""
    if df_fc is None or df_fc.empty:
        return pd.DataFrame(columns=["Symbol","Date","Series","Value","Lower","Upper"])
    d = df_fc.rename(columns={"Forecast":"Value"}).copy()
    d["Series"] = "Forecast"
    return d[["Symbol","Date","Series","Value","Lower","Upper"]]

def sanitize(df: pd.DataFrame | None) -> pd.DataFrame | None:
    """Drop all-NA rows/cols; ensure Value column exists and has any non-NA."""
    if df is None:
        return None
    d = df.copy()
    d = d.dropna(how="all")           # remove empty rows
    d = d.dropna(axis=1, how="all")   # remove empty cols
    if d.empty:
        return None
    if "Value" in d.columns and not d["Value"].notna().any():
        return None
    return d

# ---------------- monthly histories ----------------
tsla_hist = to_monthly_close(tesla_data, "TSLA")
gme_hist  = to_monthly_close(gme_data,  "GME")
tsla_hist.to_csv(out_data / "TSLA_monthly_history.csv", index=False)
gme_hist.to_csv(out_data / "GME_monthly_history.csv", index=False)

# ---------------- forecasts (use variables if computed earlier) ----------------
try:
    tsla_fc = fmt_forecast(forecast_df, "TSLA")
except NameError:
    tsla_fc = fmt_forecast(None, "TSLA")
tsla_fc.to_csv(out_data / "TSLA_forecast_monthly.csv", index=False)

try:
    gme_fc = fmt_forecast(forecast_gme, "GME")
except NameError:
    gme_fc = fmt_forecast(None, "GME")
gme_fc.to_csv(out_data / "GME_forecast_monthly.csv", index=False)

# ---------------- Tableau long-format (strict, warning-free) ----------------
frames = [
    sanitize(actual_long(tsla_hist)),
    sanitize(actual_long(gme_hist)),
    sanitize(forecast_long(tsla_fc)),
    sanitize(forecast_long(gme_fc)),
]
frames = [f for f in frames if f is not None]

if frames:
    tableau_long = pd.concat(frames, ignore_index=True)
    tableau_long = tableau_long.sort_values(["Symbol","Date","Series"]).reset_index(drop=True)
    tableau_long.to_csv(out_data / "Stocks_TSLA_GME_monthly_actual_forecast_long.csv", index=False)
else:
    print("⚠️ No valid DataFrames to concatenate for Tableau long-format.")

# ---------------- HTML figures: Q5/Q6 dashboards ----------------
# TSLA stock vs revenue
try:
    ts = tesla_data.sort_values("Date").copy()
    ts["Date"] = pd.to_datetime(ts["Date"], errors="coerce", utc=True).dt.tz_localize(None)
    tr = tesla_revenue.sort_values("Date").copy()
    tr["Date"] = pd.to_datetime(tr["Date"], errors="coerce")
    tr["Revenue"] = pd.to_numeric(tr["Revenue"], errors="coerce")

    fig_tsla = go.Figure()
    fig_tsla.add_trace(go.Scatter(x=ts["Date"], y=ts["Close"], mode="lines", name="Tesla Close"))
    fig_tsla.add_trace(go.Scatter(x=tr["Date"], y=tr["Revenue"], mode="lines+markers", name="Tesla Revenue", yaxis="y2"))
    fig_tsla.update_layout(
        title="Tesla: Stock Price vs Quarterly Revenue",
        xaxis_title="Date",
        yaxis=dict(title="Close (USD)"),
        yaxis2=dict(title="Revenue (USD)", overlaying="y", side="right"),
        xaxis_rangeslider=dict(visible=False),
        height=650
    )
    fig_tsla.write_html(out_figs / "tesla_graph.html", include_plotlyjs="cdn")
except Exception as _:
    pass

# GME stock vs revenue
try:
    gs = gme_data.sort_values("Date").copy()
    gs["Date"] = pd.to_datetime(gs["Date"], errors="coerce", utc=True).dt.tz_localize(None)
    gr = gme_revenue.sort_values("Date").copy()
    gr["Date"] = pd.to_datetime(gr["Date"], errors="coerce")
    gr["Revenue"] = pd.to_numeric(gr["Revenue"], errors="coerce")

    fig_gme = go.Figure()
    fig_gme.add_trace(go.Scatter(x=gs["Date"], y=gs["Close"], mode="lines", name="GameStop Close"))
    fig_gme.add_trace(go.Scatter(x=gr["Date"], y=gr["Revenue"], mode="lines+markers", name="GameStop Revenue", yaxis="y2"))
    fig_gme.update_layout(
        title="GameStop: Stock Price vs Quarterly Revenue",
        xaxis_title="Date",
        yaxis=dict(title="Close (USD)"),
        yaxis2=dict(title="Revenue (USD)", overlaying="y", side="right"),
        xaxis_rangeslider=dict(visible=False),
        height=650
    )
    fig_gme.write_html(out_figs / "gme_graph.html", include_plotlyjs="cdn")
except Exception as _:
    pass

# ---------------- HTML figures: Forecast plots (if forecasts exist) ----------------
# TSLA forecast
try:
    dfp_tsla = tsla_hist.rename(columns={"Date":"ds","Close":"y"})[["ds","y"]].copy()
    fig_tfc = go.Figure()
    fig_tfc.add_trace(go.Scatter(x=dfp_tsla["ds"], y=dfp_tsla["y"], mode="lines", name="TSLA Close (actual)"))
    if not tsla_fc.empty:
        fig_tfc.add_trace(go.Scatter(x=tsla_fc["Date"], y=tsla_fc["Forecast"], mode="lines+markers", name="Forecast"))
        fig_tfc.add_trace(go.Scatter(
            x=tsla_fc["Date"].tolist() + tsla_fc["Date"].tolist()[::-1],
            y=tsla_fc["Upper"].tolist() + tsla_fc["Lower"].tolist()[::-1],
            fill="toself", opacity=0.2, line=dict(width=0), name="Confidence Interval"
        ))
    fig_tfc.update_layout(title="TSLA Monthly Forecast", xaxis_title="Date", yaxis_title="Price (USD)", height=600)
    fig_tfc.write_html(out_figs / "tsla_forecast.html", include_plotlyjs="cdn")
except Exception as _:
    pass

# GME forecast
try:
    dfp_gme_plot = gme_hist.rename(columns={"Date":"ds","Close":"y"})[["ds","y"]].copy()
    fig_gfc = go.Figure()
    fig_gfc.add_trace(go.Scatter(x=dfp_gme_plot["ds"], y=dfp_gme_plot["y"], mode="lines", name="GME Close (actual)"))
    if not gme_fc.empty:
        fig_gfc.add_trace(go.Scatter(x=gme_fc["Date"], y=gme_fc["Forecast"], mode="lines+markers", name="Forecast"))
        fig_gfc.add_trace(go.Scatter(
            x=gme_fc["Date"].tolist() + gme_fc["Date"].tolist()[::-1],
            y=gme_fc["Upper"].tolist() + gme_fc["Lower"].tolist()[::-1],
            fill="toself", opacity=0.2, line=dict(width=0), name="Confidence Interval"
        ))
    fig_gfc.update_layout(title="GME Monthly Forecast", xaxis_title="Date", yaxis_title="Price (USD)", height=600)
    fig_gfc.write_html(out_figs / "gme_forecast.html", include_plotlyjs="cdn")
except Exception as _:
    pass

# ---------------- report ----------------
print("Saved CSVs:", *(p.name for p in sorted(out_data.glob('*.csv'))), sep="\n - ")
print("\nSaved HTML figures:", *(p.name for p in sorted(out_figs.glob('*.html'))), sep="\n - ")



Saved CSVs:
 - GME_forecast_monthly.csv
 - GME_monthly_history.csv
 - Stocks_TSLA_GME_monthly_actual_forecast_long.csv
 - TSLA_forecast_monthly.csv
 - TSLA_monthly_history.csv

Saved HTML figures:
 - gme_forecast.html
 - gme_graph.html
 - tesla_graph.html
 - tsla_forecast.html
